In [2]:
!pip install syft==0.2.9

import torch
import pandas as pd
import numpy as np
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVR
from sklearn.metrics import mean_absolute_error


In [3]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

<VirtualWorker id:secure_worker #objects:0>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from sklearn.svm import LinearSVR


In [ ]:
dataframe=np.genfromtxt("/content/drive/MyDrive/IIIT-K_Internship/Diabetes-FL/diabetes1.csv",delimiter=',',skip_header=1)

In [ ]:
X=dataframe[:,0:8]
Y=dataframe[:,8]
Y=Y.reshape(768,1)
dtype=torch.float
device=torch.device("cpu")

In [ ]:
from sklearn.preprocessing import StandardScaler
model=StandardScaler()
X=model.fit_transform(X)


### Mobility data

In [6]:
df1 = pd.read_csv('/content/drive/MyDrive/IIIT-K_Internship/Mobility-prediction/data/vanet-trace-creteil-20130924-0700-0900.csv', sep=';')
df2 = pd.read_csv('/content/drive/MyDrive/IIIT-K_Internship/Mobility-prediction/data/vanet-trace-creteil-20130924-1700-1900.csv',sep=';')

In [ ]:
len(df1)

962721

In [7]:
df1=df1[df1['vehicle_id']=='BusFlowNorthSouth0.0']

cols = ['vehicle_slope', 'vehicle_lane', 'vehicle_angle', 'vehicle_type', 'vehicle_pos', 'vehicle_speed','vehicle_id', 'vehicle_y']
df1.drop(cols, axis=1, inplace=True)

In [ ]:
len(df1)

180

### Federated - Latest

In [8]:
df2 = df1.copy()
df2 = df2.reset_index()

### *quarter-split*

In [9]:
X_train = df2['timestep_time'][:45]
y_train = df2['vehicle_x'][:45]
X_train = X_train.append(df2['timestep_time'][90:135])
y_train = y_train.append(df2['vehicle_x'][90:135])


X_train1 = df2['timestep_time'][45:90]
y_train1 = df2['vehicle_x'][45:90]
X_train1 = X_train1.append(df2['timestep_time'][135:180])
y_train1 = y_train1.append(df2['vehicle_x'][135:180])

### *half-split*

In [27]:
from sklearn.model_selection import train_test_split

X_train = df2['timestep_time'][:int(len(df2)*0.8)]
y_train = df2['vehicle_x'][:int(len(df2)*0.8)]
X_test = df2['timestep_time'][int(len(df2)*0.8):]
y_test = df2['vehicle_x'][int(len(df2)*0.8):]

In [28]:
time_st = X_train.to_numpy()
vehicle_x = y_train.to_numpy()

#Convert to 1d Vector
time_st = np.reshape(time_st, (len(time_st), 1))
vehicle_x = np.reshape(vehicle_x, (len(vehicle_x), 1))

In [10]:
time_st1 = X_train.to_numpy()
vehicle_x1 = y_train.to_numpy()

#Convert to 1d Vector
time_st1 = np.reshape(time_st1, (len(time_st1), 1))
vehicle_x1 = np.reshape(vehicle_x1, (len(vehicle_x1), 1))



In [11]:
time_st2 = X_train1.to_numpy()
vehicle_x2 = y_train1.to_numpy()

#Convert to 1d Vector
time_st2 = np.reshape(time_st2, (len(time_st2), 1))
vehicle_x2 = np.reshape(vehicle_x2, (len(vehicle_x2), 1))

In [ ]:
time_st1 = time_st[:int(len(time_st)/2)]
time_st2 = time_st[int(len(time_st)/2):]
vehicle_x1 = vehicle_x[:int(len(vehicle_x)/2)]
vehicle_x2 = vehicle_x[int(len(vehicle_x)/2):]

In [ ]:
time_st2

array([[ 45.],
       [ 46.],
       [ 47.],
       [ 48.],
       [ 49.],
       [ 50.],
       [ 51.],
       [ 52.],
       [ 53.],
       [ 54.],
       [ 55.],
       [ 56.],
       [ 57.],
       [ 58.],
       [ 59.],
       [ 60.],
       [ 61.],
       [ 62.],
       [ 63.],
       [ 64.],
       [ 65.],
       [ 66.],
       [ 67.],
       [ 68.],
       [ 69.],
       [ 70.],
       [ 71.],
       [ 72.],
       [ 73.],
       [ 74.],
       [ 75.],
       [ 76.],
       [ 77.],
       [ 78.],
       [ 79.],
       [ 80.],
       [ 81.],
       [ 82.],
       [ 83.],
       [ 84.],
       [ 85.],
       [ 86.],
       [ 87.],
       [ 88.],
       [ 89.],
       [135.],
       [136.],
       [137.],
       [138.],
       [139.],
       [140.],
       [141.],
       [142.],
       [143.],
       [144.],
       [145.],
       [146.],
       [147.],
       [148.],
       [149.],
       [150.],
       [151.],
       [152.],
       [153.],
       [154.],
       [155.],
       [15

In [12]:
#svr_rbf = LinearSVR(C= 1e3)
#svr_rbf.fit(time_st, vehicle_x)


from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
gsc1 = GridSearchCV(
        estimator=LinearSVR(),
        param_grid={
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
            #'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
'''
param_grid={
            'C': [1.1, 5.4, 170, 1001],
            'epsilon': [0.0003, 0.007, 0.0109, 0.019, 0.14, 0.05, 8, 0.2, 3, 2, 7],
            'gamma': [0.7001, 0.008, 0.001, 3.1, 1, 1.3, 5]
        }
'''

"\nparam_grid={\n            'C': [1.1, 5.4, 170, 1001],\n            'epsilon': [0.0003, 0.007, 0.0109, 0.019, 0.14, 0.05, 8, 0.2, 3, 2, 7],\n            'gamma': [0.7001, 0.008, 0.001, 3.1, 1, 1.3, 5]\n        }\n"

In [20]:
grid_result1 = gsc1.fit(time_st1, vehicle_x1.ravel())
best_params1 = grid_result1.best_params_
print(best_params1)

{'C': 100, 'epsilon': 0.1}


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
gsc2 = GridSearchCV(
        estimator=LinearSVR(),#SVR(kernel='rbf'),
        param_grid={
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
            #'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result2 = gsc2.fit(time_st2, vehicle_x2.ravel())
best_params2 = grid_result2.best_params_
print(best_params2)

{'C': 100, 'epsilon': 0.0005}


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [29]:
C_best = (best_params1['C']+best_params2['C'])/2
epsilon_best = (best_params1['epsilon']+best_params2['epsilon'])/2
#gamma_best = (best_params1['gamma']+best_params2['gamma'])/2

In [30]:
final_svr = LinearSVR(C = C_best, epsilon=epsilon_best)#, gamma = gamma_best)#SVR(kernel='rbf',C = C_best, epsilon=epsilon_best, gamma = gamma_best)
final_svr.fit(time_st, vehicle_x.ravel())


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=100.0, dual=True, epsilon=0.05025, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [31]:
time_st_test = X_test.to_numpy()
time_st_test = np.reshape(time_st_test, (len(time_st_test), 1))

In [32]:
pred = final_svr.predict(time_st_test)

In [33]:
y_expected = y_test.to_numpy()

In [34]:
pred

array([1194.13296042, 1193.85470737, 1193.57645433, 1193.29820128,
       1193.01994824, 1192.74169519, 1192.46344215, 1192.1851891 ,
       1191.90693606, 1191.62868301, 1191.35042997, 1191.07217692,
       1190.79392388, 1190.51567083, 1190.23741779, 1189.95916474,
       1189.6809117 , 1189.40265865, 1189.12440561, 1188.84615256,
       1188.56789952, 1188.28964647, 1188.01139343, 1187.73314038,
       1187.45488734, 1187.17663429, 1186.89838125, 1186.6201282 ,
       1186.34187516, 1186.06362211, 1185.78536907, 1185.50711602,
       1185.22886298, 1184.95060993, 1184.67235689, 1184.39410384])

In [35]:
y_expected

array([1197.7 , 1197.7 , 1197.7 , 1197.7 , 1197.7 , 1197.7 , 1197.7 ,
       1197.7 , 1197.7 , 1197.7 , 1197.7 , 1197.7 , 1197.7 , 1197.7 ,
       1197.17, 1196.13, 1196.3 , 1198.78, 1199.85, 1199.85, 1199.85,
       1199.85, 1199.85, 1199.85, 1199.92, 1201.69, 1205.59, 1204.5 ,
       1200.  , 1196.41, 1192.98, 1189.16, 1184.89, 1180.81, 1177.71,
       1174.24])

In [36]:
from sklearn.metrics import classification_report

from sklearn.metrics import mean_squared_error

print(mean_squared_error(y_expected, pred))

82.96431363052297


In [38]:
print("Root mean square error - RMSE ") 
print(mean_squared_error(y_expected, pred, squared=False))

Root mean square error - RMSE 
9.108474824608287


In [39]:
print("Mean absolute error - MAE")
print(mean_absolute_error(y_expected, pred))

Mean absolute error - MAE
8.135686406725528


In [40]:
from sklearn.utils.validation import check_consistent_length, check_array

def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):
    """Mean absolute percentage error regression loss.
    Note here that we do not represent the output as a percentage in range
    [0, 100]. Instead, we represent it in range [0, 1/eps]. Read more in the
    :ref:`User Guide <mean_absolute_percentage_error>`.
    .. versionadded:: 0.24
    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.
    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.
    multioutput : {'raw_values', 'uniform_average'} or array-like
        Defines aggregating of multiple output values.
        Array-like value defines weights used to average errors.
        If input is list then the shape must be (n_outputs,).
        'raw_values' :
            Returns a full set of errors in case of multioutput input.
        'uniform_average' :
            Errors of all outputs are averaged with uniform weight.
    Returns
    -------
    loss : float or ndarray of floats in the range [0, 1/eps]
        If multioutput is 'raw_values', then mean absolute percentage error
        is returned for each output separately.
        If multioutput is 'uniform_average' or an ndarray of weights, then the
        weighted average of all output errors is returned.
        MAPE output is non-negative floating point. The best value is 0.0.
        But note the fact that bad predictions can lead to arbitarily large
        MAPE values, especially if some y_true values are very close to zero.
        Note that we return a large value instead of `inf` when y_true is zero.
    Examples
    --------
    >>> from sklearn.metrics import mean_absolute_percentage_error
    >>> y_true = [3, -0.5, 2, 7]
    >>> y_pred = [2.5, 0.0, 2, 8]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    0.3273...
    >>> y_true = [[0.5, 1], [-1, 1], [7, -6]]
    >>> y_pred = [[0, 2], [-1, 2], [8, -5]]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    0.5515...
    >>> mean_absolute_percentage_error(y_true, y_pred, multioutput=[0.3, 0.7])
    0.6198...
    """
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

def _check_reg_targets(y_true, y_pred, multioutput, dtype="numeric"):
    """Check that y_true and y_pred belong to the same regression task.
    Parameters
    ----------
    y_true : array-like
    y_pred : array-like
    multioutput : array-like or string in ['raw_values', uniform_average',
        'variance_weighted'] or None
        None is accepted due to backward compatibility of r2_score().
    Returns
    -------
    type_true : one of {'continuous', continuous-multioutput'}
        The type of the true target data, as output by
        'utils.multiclass.type_of_target'.
    y_true : array-like of shape (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples, n_outputs)
        Estimated target values.
    multioutput : array-like of shape (n_outputs) or string in ['raw_values',
        uniform_average', 'variance_weighted'] or None
        Custom output weights if ``multioutput`` is array-like or
        just the corresponding argument if ``multioutput`` is a
        correct keyword.
    dtype : str or list, default="numeric"
        the dtype argument passed to check_array.
    """
    check_consistent_length(y_true, y_pred)
    y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)

    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))

    if y_pred.ndim == 1:
        y_pred = y_pred.reshape((-1, 1))

    if y_true.shape[1] != y_pred.shape[1]:
        raise ValueError("y_true and y_pred have different number of output "
                         "({0}!={1})".format(y_true.shape[1], y_pred.shape[1]))

    n_outputs = y_true.shape[1]
    allowed_multioutput_str = ('raw_values', 'uniform_average',
                               'variance_weighted')
    if isinstance(multioutput, str):
        if multioutput not in allowed_multioutput_str:
            raise ValueError("Allowed 'multioutput' string values are {}. "
                             "You provided multioutput={!r}".format(
                                 allowed_multioutput_str,
                                 multioutput))
    elif multioutput is not None:
        multioutput = check_array(multioutput, ensure_2d=False)
        if n_outputs == 1:
            raise ValueError("Custom weights are useful only in "
                             "multi-output cases.")
        elif n_outputs != len(multioutput):
            raise ValueError(("There must be equally many custom weights "
                              "(%d) as outputs (%d).") %
                             (len(multioutput), n_outputs))
    y_type = 'continuous' if n_outputs == 1 else 'continuous-multioutput'

    return y_type, y_true, y_pred, multioutput

In [41]:
print("MAPE")
print(mean_absolute_percentage_error(y_expected, pred))

MAPE
0.006793146924167424


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')



test_set_range = df2[int(len(df1)*0.8):].index
plt.plot(X_test, pred, color='blue', marker='o', linestyle='dashed',label='Predicted')
plt.plot(X_test, y_expected, color='red', label='Actual')
plt.title('X prediction')
plt.xlabel('Vehicle - x')
plt.ylabel('Timestamp')
#plt.xticks(np.arange(881,1259,50), df.Date[881:1259:50])
plt.legend()
plt.show()

### Federated - Old

In [ ]:
df1=df1.to_numpy()

In [ ]:
X=df1[:,0]
Y=df1[:,1]
X=X.reshape(180,1)
Y=Y.reshape(180,1)
dtype=torch.float
device=torch.device("cpu")

In [ ]:

X_train =X[:int(len(df1)*0.8)]
y_train =Y[:int(len(df1)*0.8)]
X_test = X[int(len(df1)*0.8):]
y_test = Y[int(len(df1)*0.8):]

In [ ]:
'''

X_train = df2['timestep_time'][:int(len(df2)*0.8)]
y_train = df2['vehicle_x'][:int(len(df2)*0.8)]
X_test = df2['timestep_time'][int(len(df2)*0.8):]
y_test = df2['vehicle_x'][int(len(df2)*0.8):]

'''

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
data = torch.from_numpy(X_train)
target = torch.from_numpy(y_train)
data,target=data.type(torch.FloatTensor),target.type(torch.FloatTensor)

In [ ]:
data_length,data_width=data.shape
partition=0.01 #Input only from 0 to 0.5 for bobs data. Alice is 1- partition

In [ ]:
bobs_data = data[0:int(data_length*partition)].send(bob)
bobs_target = target[0:int(data_length*partition)].send(bob)

alices_data = data[int(data_length*partition):].send(alice)
alices_target = target[int(data_length*partition):].send(alice)


In [ ]:
# Iniitalize A Toy Model
#model = nn.Linear(8,1)
#loss= torch.nn.MSELoss()

In [ ]:
'''
class my_network(torch.nn.Module):
    def __init__(self):
        super(my_network, self).__init__()
        self.fc1=nn.Linear(data_width,200)
        self.activ=nn.ReLU()
        self.fc2=nn.Linear(200,100)
        self.activ2=nn.ReLU()
        self.fc3=nn.Linear(100,1)
#         self.activ3=nn.ReLU()
    
    def forward(self,input_):
        a1=self.fc1(input_)
        a1=self.activ(a1)
        a2=self.fc2(a1)
        a2=self.activ2(a2)
        y=self.fc3(a2)
#         y=self.activ3(y)
        return y
'''
from sklearn.svm import SVR

model= SVR(kernel= 'linear', C= 1e3, gamma= 0.1)
#svr_rbf.fit(time_st, vehicle_x)


In [ ]:
#model=my_network()
#loss= torch.nn.MSELoss()

In [ ]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

#bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
#alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

In [ ]:
'''
for i in range(1000):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = loss(bobs_pred,bobs_target)
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = loss(alices_pred,alices_target)
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
#     print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

'''

In [ ]:
bobs_model.fit()
alices_model.fit()

In [ ]:
iterations = 10
worker_iters = 200

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model.forward(bobs_data)
        bobs_loss = loss(bobs_pred,bobs_target)
        bobs_loss.backward()        
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model.forward(alices_data)
        alices_loss = loss(alices_pred,alices_target)
        alices_loss.backward()
        alices_opt.step()
#         alices_opt.step(alices_data.shape[0])
        alices_loss = alices_loss.get().data
#         print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
      model.fc1.weight.set_(((alices_model.fc1.weight.data *(1- partition) + bobs_model.fc1.weight.data * partition)).get())
      model.fc1.bias.set_(((alices_model.fc1.bias.data *(1- partition) + bobs_model.fc1.bias.data * partition)).get())
      model.fc2.weight.set_(((alices_model.fc2.weight.data *(1- partition) + bobs_model.fc2.weight.data * partition) ).get())
      model.fc2.bias.set_(((alices_model.fc2.bias.data *(1- partition) + bobs_model.fc2.bias.data * partition) ).get())
      model.fc3.weight.set_(((alices_model.fc3.weight.data *(1- partition) + bobs_model.fc3.weight.data * partition) ).get())
      model.fc3.bias.set_(((alices_model.fc3.bias.data *(1- partition) + bobs_model.fc3.bias.data * partition) ).get())

      #model.weight.set_(((alices_model.weight.data *(1- partition) + bobs_model.weight.data * partition) ).get())
      #model.bias.set_(((alices_model.bias.data *(1- partition) + bobs_model.bias.data * partition) ).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0514) Alice:tensor(0.1930)
Bob:tensor(0.0346) Alice:tensor(0.1769)
Bob:tensor(0.0278) Alice:tensor(0.1702)
Bob:tensor(0.0246) Alice:tensor(0.1661)
Bob:tensor(0.0228) Alice:tensor(0.1633)
Bob:tensor(0.0216) Alice:tensor(0.1612)
Bob:tensor(0.0208) Alice:tensor(0.1595)
Bob:tensor(0.0205) Alice:tensor(0.1580)
Bob:tensor(0.0203) Alice:tensor(0.1568)
Bob:tensor(0.0202) Alice:tensor(0.1556)


In [ ]:
svr_linear.coef_.set_()

In [ ]:

from sklearn.metrics import mean_squared_error

print(mean_squared_error(y_expected, pred))

In [ ]:
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
X_test,y_test=X_test.type(torch.FloatTensor),y_test.type(torch.FloatTensor)
pred=model(X_test)

In [ ]:
pred=torch.abs(torch.round((pred)))
output=pred-y_test

In [ ]:
incorrect_labels=len(np.nonzero(output))

accuracy=1-(incorrect_labels/len(X_test))

accuracy

0.8246753246753247

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.detach().numpy(),pred.detach().numpy()))

              precision    recall  f1-score   support

         0.0       0.84      0.93      0.88       107
         1.0       0.78      0.60      0.67        47

    accuracy                           0.82       154
   macro avg       0.81      0.76      0.78       154
weighted avg       0.82      0.82      0.82       154

